In [ ]:
# all necessary requestcompletion imports 
import src.requestcompletion as rc
from src.requestcompletion.llm import MessageHistory, UserMessage, SystemMessage
from src.requestcompletion.visuals.agent_viewer import AgentViewer
from src.requestcompletion.nodes.library import from_function

<img src="./assets/Telus-Symbol.png" alt="Telus Symbol" width="200"/>

## RC Demo: Customer Service Ticketing Agent + Notion:

In this demo we will demonstrate how to use RequestCompletion to built AI Agents and combine them to perrform complex tasks using just a few lines of code.

### Step 1: Import and use Notion Agent from sample_agents

In [ ]:
from demo.sample_agents import NotionAgent

### Step 2: Invoke the agent 🚀

In [ ]:
USER_MESSAGE = UserMessage(...)

In [ ]:
# construct message history 
mess_hist = ...

# Run the agent
with rc.Runner() as runner:
    result = await runner.run(..., mess_hist)

we can also visualize the agent control flow

In [ ]:
viewer = AgentViewer(stamps=result.all_stamps,
                     node_heap=result.node_heap,
                     request_heap=result.request_heap)

viewer.display_graph()

## Now lets build our own Agent for Telus

We have some mock functions that we will give the Agent access to as tools.

In [ ]:
# imports
from demo.sample_tools.notion_tools import NotionAgenticTool
from demo.sample_tools.telus_tools import get_service_tickets, create_service_ticket, check_network_status, analyze_call_volume

In [ ]:
# create randomized mock data (mock telus Database)
mock_entries = [
    ("internet_outage", "Vancouver, BC"),
    ("billing", "Vancouver, BC"),
    ("device_support", "Langley, BC"),
    ("internet_outage", "Burnbay, BC"),
    ("tv_service", "Vancouver, BC"),
    ("mobile_service", "Burnaby, BC"),
    ("billing", "Kelowna, BC"),
    ("fiber_installation", "Richmond, BC"),
    ("device_support", "Victoria, BC"),
    ("network_latency", "Abbotsford, BC")
]

# Populate the mock database
for issue_type, location in mock_entries:
    result = create_service_ticket(issue_type, location)

### Step 1: Prompts for the Agent

In [ ]:
SYSTEM_PROMPT_TELUS = SystemMessage("You are a helpful Telus agent. You have access to the following tools: check_network_status," \
"analyze_call_volume, create_service_ticket, get_service_tickets, and NotionTool. Your goal is to resolve the user's request as efficiently as possible." \
"Today is April 28th 2025.")

In [ ]:
USER_MESSAGE = UserMessage(...)

### Step 2: Construct the Agent

In [ ]:
TelusTicketingAgent = ...

### Step 3: Invoke the Agent 🚀

In [ ]:
# See Step 2 for Notion Agent
...

In [ ]:
viewer = AgentViewer(stamps=result.all_stamps,
                     node_heap=result.node_heap,
                     request_heap=result.request_heap)

viewer.display_graph()